In [6]:
from torchvision import models
import torch.utils.data.dataset
import torch
import torch.nn as nn
import torch.optim as optimize
import image_loader as il
import properties as pr
from torch.multiprocessing import Pool
import os 

device = 'cuda' if torch.cuda.is_available() else "cpu"

if device == 'cuda':
    torch.multiprocessing.set_start_method('spawn', force=True)
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
                               
print(torch.cuda.is_available())
print( torch.cuda.get_device_name(0))

if os.path.isdir(pr.tensor_train_path) and os.path.isdir(pr.tensor_validate_path):
    print("prepared data is exists")
    train_tensor = il.load_tensors(pr.tensor_train_path)
    validate_tensor = il.load_tensors(pr.tensor_validate_path)
else:
    print("prepared data doesn't exists")
    r = il.map_inputs_labels(pr.data_inputs_path, pr.data_labels_path)
    train, validate = il.split_set(r, pr.train_size, pr.test_size)
    il.prepare_data(train, validate)
    for i in train:
        il.save_data(pr.tensor_train_path, i)
    for j in validate:
        il.save_data(pr.tensor_validate_path, j)
        
    train_tensor = il.load_tensors(pr.tensor_train_path)
    validate_tensor = il.load_tensors(pr.tensor_validate_path)
"""
train_tensor = list(map(il.apply_resize, train_tensor))
validate_tensor = list(map(il.apply_resize, validate_tensor))

for i in train_tensor:
    il.save_data(pr.tensor_train_path, i)
for j in validate_tensor:
    il.save_data(pr.tensor_validate_path, j)
"""

train_tensor = list(map(il.remove_empty_mask, train_tensor))
validate_tensor = list(map(il.remove_empty_mask, validate_tensor))

True
GeForce GTX 1060 with Max-Q Design
prepared data is exists


In [2]:
def get_model_parameters_to_update(m:models.AlexNet) -> list:
    params = []
    for name, param in m.named_parameters():
        # maybe here better check require gradient back propagation
        # and then update
        # print("name={}".format(name))
        params.append(param)
    return params

def set_model_last_layer(m:models.AlexNet) -> models.AlexNet:
    num_features = m.classifier[6].in_features
    m.classifier[6] = nn.Linear(num_features, pr.labels_number) # here 2 for each label 
    return m

In [3]:
### EXECUTE ONLY ONCE!!!!!!!!!!!
model = models.alexnet(pretrained=True, progress=False)
model = set_model_last_layer(model)

parameters = get_model_parameters_to_update(model)
data_train = torch.utils.data.DataLoader(il.CustomDataset(train_tensor), batch_size=4, shuffle=True, num_workers=4)
data_validate = torch.utils.data.DataLoader(il.CustomDataset(validate_tensor), batch_size=4, shuffle=True, num_workers=4)
crit = nn.SmoothL1Loss()
optim = optimize.SGD(parameters, lr=0.001, momentum=0.9)


In [4]:
from model_processing.train_model import TrainModel

from model_processing.validate_model import ValidateModel

In [5]:
validate_model = ValidateModel(pr.labels_number, device)
train_model = TrainModel(validate_model, device)

train_model.train_model(model, data_train,data_validate, crit, optim, 10)

Epoch: 1/10
acc: 0.9158437994538648
--------------------
Epoch: 2/10
acc: 0.9045918345978704
--------------------
Epoch: 3/10
acc: 0.9134134785693929
--------------------
Epoch: 4/10
acc: 0.9201382334890909
--------------------
Epoch: 5/10
acc: 0.9065144023935922
--------------------
Epoch: 6/10
acc: 0.9097917530667047
--------------------
Epoch: 7/10
acc: 0.9147743398809975
--------------------
Epoch: 8/10
acc: 0.8996274051075769
--------------------
Epoch: 9/10
acc: 0.9127863482955301
--------------------
Epoch: 10/10
acc: 0.9159036728674325
--------------------
best acc: {} 0.9201382334890909


<All keys matched successfully>